In [1]:
! pip install pylibjpeg pylibjpeg-libjpeg pydicom
! pip install -U python-gdcm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 34.9 MB/s eta 0:00:0000:0100:01


<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1 Libraries</b></p>
</div>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc
import gdcm
import pydicom
import joblib
from pydicom import dcmread
import pylibjpeg
from pydicom.pixel_data_handlers.util import apply_voi_lut
import scipy.ndimage
from tqdm import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import warnings
import dask.array as da
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, callbacks
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image as im
from keras.models import load_model
from random import shuffle
import dask
import sys
%matplotlib inline
sns.set(style='darkgrid', font_scale=1.6)

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>2 Activating Devices</b></p>
</div>

In [3]:
DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Using default strategy for CPU and single GPU
Num GPUs Available:  1


2022-09-08 12:21:46.737876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 12:21:46.744824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 12:21:46.746024: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>3 Data</b></p>
</div>


In [4]:
#Loading Scans
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# Load the scans in given folder path
def load_scan(path):
    
    dcm_paths = glob(f"{path}/*")
    dcm_paths.sort(key=natural_keys)
    
    patient_scan = [pydicom.dcmread(paths) for paths in dcm_paths]
    
    return patient_scan

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4 Preprocessing</b></p>
</div>


<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.1 Loding and Conversion to HU</b></p>
</div>

In [5]:
def get_pixels_hu(slices):
   
    image = np.stack([cv2.resize(s.pixel_array,(512,512),interpolation = cv2.INTER_NEAREST) for s in slices])
    
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    image = da.from_array(image) #Using Dask to speed up processing
    
    
    # Convert to Hounsfield units (HU)
        
    intercept = da.from_array([slices[slice_number].RescaleIntercept for slice_number in range(len(slices))])
    slope = da.from_array([slices[slice_number].RescaleSlope for slice_number in range(len(slices))])
    
    intercept=intercept.reshape((-1,1,1))
    slope=slope.reshape((-1,1,1))
    
    image= slope * image.astype("float32")
        
    image+= intercept
    
    return image.astype("int16")

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.2 Normalization</b></p>
</div>

As HU values range from **150** to **2050**. So we will use this values for normalization.

In [6]:
# https://radiopaedia.org/articles/windowing-ct
W=1800
L=400
def CT_windowing(image):
    image = (image-L+0.5*W)/W
    image[image<0] = 0
    image[image>1] = 1
    
    return image

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>5 Model</b></p>
</div>

In [7]:
def non_segment():
    train_paths = glob(f"../input/rsna-2022-cervical-spine-fracture-detection/train_images/*")
    train_patient=[]
    for path in train_paths:
        train_patient.append((path.rsplit("/",1)[-1]))#Patient with mask present

    seg_paths = glob(f"../input/rsna-2022-cervical-spine-fracture-detection/segmentations/*")
    seg_patient=[]
    for path in seg_paths:
        seg_patient.append((path.rsplit("/",1)[-1][:-4]))#Patient with mask present

    bounding_boxes=pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
    patient=bounding_boxes["StudyInstanceUID"].unique()
    
    not_seg_patient=[]
    for i in train_patient:
        if i not in seg_patient and i in patient:
            not_seg_patient.append(i)

    return not_seg_patient

In [8]:
not_seg_patient=non_segment()
len(not_seg_patient)

195

In [9]:
gc.collect()

49

In [10]:
def preprocess(i):
    path_scan=f"../input/rsna-2022-cervical-spine-fracture-detection/train_images/{i}"
    
    image=load_scan(path_scan)
    image=get_pixels_hu(image)
    image=CT_windowing(image)
    image=da.array(image)
    
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        image=np.reshape(image,(-1,512,512,1))
    
    return image

In [11]:
def main_func(i,csv):
    
    model = keras.models.load_model("../input/classify/Classifier.h5.zip")
    
    image = preprocess(not_seg_patient[i])
    
    csv.append(model.predict(np.reshape(image,(-1,512,512,1))).astype('float16'))
    gc.collect()
    
    tf.keras.backend.clear_session()
    gc.collect()

In [12]:
def built_csv():
    csv = []
    for i in tqdm(range(len(not_seg_patient))):
        main_func(i,csv)
        gc.collect() 
        
    return csv

In [ ]:
csv=built_csv()

  0%|          | 0/195 [00:00<?, ?it/s]2022-09-08 12:21:47.330247: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-08 12:21:47.330708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 12:21:47.331586: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 12:21:47.332243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at 

In [ ]:
from csv import writer
filename = 'pred.csv'
with open(filename, 'a') as f_object: 
        writer_object = writer(f_object)
        writer_object.writerows()
        f_object.close() 

In [ ]:
os.remove("./pred.csv")